In [86]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
import pandahouse
from read_db.CH import Getch
import os
import datetime
from datetime import datetime
from datetime import date, timedelta

In [103]:
bot = telegram.Bot(token='')
chat_id = ''

### Получение данных

In [88]:
value_DAU=Getch('''select count(DISTINCT(user_id)) 
             from simulator_20220220.feed_actions where toDate(time) = today()-1
            ''').df

df_DAU_7=Getch('''select user_id, time
             from simulator_20220220.feed_actions where toDate(time) < today() and toDate(time) > today()-8
            ''').df

In [89]:
value_View=Getch('''select count(user_id) 
             from simulator_20220220.feed_actions where toDate(time) = today()-1
             and action='view'
            ''').df

df_view_7=Getch('''select user_id, time
             from simulator_20220220.feed_actions where toDate(time) < today() and toDate(time) > today()-8
             and action='view'
            ''').df

In [90]:
value_Like=Getch('''select count(user_id) 
             from simulator_20220220.feed_actions where toDate(time) = today()-1
             and action='like'
            ''').df

df_like_72=Getch('''select user_id, time
             from simulator_20220220.feed_actions where toDate(time) < today() and toDate(time) > today()-8
             and action='like'
            ''').df

In [91]:
value_CTR=value_Like/value_View

### Обработка данных

In [92]:
def get_day(df):
    df['Day']=pd.DatetimeIndex(df.time).date
    return df

In [93]:
df_DAU_7=get_day(df_DAU_7)
df_DAU_7=df_DAU_7.groupby('Day').agg({"user_id": "nunique"}).reset_index()

In [94]:
df_view_7=get_day(df_view_7)
df_view_7=df_view_7.groupby('Day').agg({"user_id": "count"}).reset_index()

In [95]:
df_like_7=get_day(df_like_72)
df_like_7=df_like_7.groupby('Day').agg({"user_id": "count"}).reset_index()

In [96]:
df_like_73=get_day(df_like_72)
df_like_73=df_like_73.groupby('Day').agg({"user_id": "count"}).reset_index()
df_CTR_7=df_like_73
df_CTR_7['user_id']=df_CTR_7['user_id']/df_view_7['user_id']

### Визуализация

In [97]:
def send_lineplot():

    sns.set()

    fig, axes = plt.subplots(2,2,figsize=(16,8))
    fig.tight_layout(pad=6.0)

    sns.lineplot(data=df_DAU_7, x='Day', y='user_id', ax=axes[0,0])
    axes[0,0].set_title('DAU',fontsize=20)
    axes[0,0].tick_params(axis='x', rotation=45,labelsize=8)
    axes[0,0].tick_params(axis='y',labelsize=8)
    axes[0,0].set(ylabel=None, xlabel=None)

    sns.lineplot(data=df_view_7, x='Day', y='user_id', ax=axes[0,1])
    axes[0,1].set_title('Просмотры',fontsize=20)
    axes[0,1].tick_params(axis='x', rotation=45,labelsize=8)
    axes[0,1].tick_params(axis='y',labelsize=8)
    axes[0,1].set(ylabel=None, xlabel=None)

    sns.lineplot(data=df_like_7, x='Day', y='user_id', ax=axes[1,0])
    axes[1,0].set_title('Лайки',fontsize=20)
    axes[1,0].tick_params(axis='x', rotation=45,labelsize=8)
    axes[1,0].tick_params(axis='y',labelsize=8)
    axes[1,0].set(ylabel=None, xlabel=None)

    sns.lineplot(data=df_CTR_7, x='Day', y='user_id', ax=axes[1,1])
    axes[1,1].set_title('CTR',fontsize=20)
    axes[1,1].tick_params(axis='x', rotation=45,labelsize=8)
    axes[1,1].tick_params(axis='y',labelsize=8)
    axes[1,1].set(ylabel=None, xlabel=None)

    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)
    plot_object.name = 'test_plot.png'
    plt.close()
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)

### Показатели за предыдущий день

In [98]:
def send_text():
    
    start='Отчет на ' + str((datetime.now()).strftime('%d-%m-%Y %H:%M:%S'))

    value_DAU_tx='{0:,}'.format(value_DAU.loc[0, 'uniqExact(user_id)']).replace(',', ' ')
    value_DAU_tx='DAU за '+str((date.today() - timedelta(days=1)).strftime('%d-%m-%Y'))+' равен '+str(value_DAU_tx)
    
    value_View_tx='{0:,}'.format(value_View.loc[0, 'count(user_id)']).replace(',', ' ')
    value_View_tx='Просмотры за '+str((date.today() - timedelta(days=1)).strftime('%d-%m-%Y'))+' равны '+str(value_View_tx)

    value_Like_tx='{0:,}'.format(value_Like.loc[0, 'count(user_id)']).replace(',', ' ')
    value_Like_tx='Лайки за '+str((date.today() - timedelta(days=1)).strftime('%d-%m-%Y'))+' равны '+str(value_Like_tx)

    value_CTR_tx=float('{:.2f}'.format(value_CTR.loc[0, 'count(user_id)']))
    value_CTR_tx='CTR за '+str((date.today() - timedelta(days=1)).strftime('%d-%m-%Y'))+' равен '+str(value_CTR_tx)

    end='Динамика показателей за последние 7 дней:'
    
    line=start+"\n"+"\n"+value_DAU_tx+"\n"+value_View_tx+"\n"+value_Like_tx+"\n"+value_CTR_tx+"\n"+"\n"+end
    
    bot.sendMessage(chat_id=chat_id,text=line)

In [99]:
def send():
    send_text()
    send_lineplot()

In [104]:
try:
    send()
except Exception as e:
    print(e)
